In [0]:
# import SparkSession from spark SQL

from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .appName('Basic Transformation - I')
    .getOrCreate()
)

In [0]:
spark

In [0]:
# emp data and schema

emp_data = [
    ['001', '101', 'Alice Johnson', '29', 'Female', '52000', '2016-03-15'],
    ['002', '102', 'Bob Smith', '34', 'Male', '48000', '2017-07-20'],
    ['003', '103', 'Charlie Brown', '26', 'Male', '47000', '2018-02-01'],
    ['004', '104', 'Diana Prince', '31', 'Female', '55000', '2015-11-12'],
    ['005', '101', 'Ethan Hunt', '28', 'Male', '49500', '2019-05-08'],
    ['006', '102', 'Fiona Gallagher', '27', 'Female', '51000', '2016-09-10'],
    ['007', '103', 'George Costanza', '33', 'Male', '46000', '2015-01-01'],
    ['008', '108', 'Hannah Montana', '25', 'Female', '53000', '2017-04-14'],
    ['009', '109', 'Isaac Newton', '35', 'Male', '60000', '2018-10-25'],
    ['010', '110', 'Jessica Jones', '32', 'Female', '58000', '2016-06-30'],
    ['010', '110', 'Jessica Jones', '32', 'Female', '58000', '2016-06-30'],
    ['011', '111', 'Kevin Malone', '36', 'Male', '49000', '2015-03-18'],
    ['012', '111', 'Laura Croft', '30', 'Female', '57000', '2017-08-22'],
    ['013', '111', 'Mike Ross', '29', 'Male', '51500', '2019-01-05'],
    ['014', '115', 'Nancy Drew', '27', 'Female', '54000', '2016-12-09'],
    ['015', '115', 'Oscar Martinez', '28', '', '45500', '2018-03-19'],
    ['016', '113', 'Pam Beesly', '31', 'Female', '47500', '2019-07-13'],
    ['017', '107', 'Quinn Fabray', '26', 'Female', '51000', '2015-11-05'],
    ['018', '109', 'Ryan Howard', '34', 'Male', '49000', '2016-04-21'],
    ['019', '109', 'Sophia Loren', '30', 'Female', '56000', '2018-09-17'],
    ['020', '120', 'Toby Flenderson', '35', 'Male', '45000', '2017-02-08'],
    ['020', '120', 'Toby Flenderson', '35', 'Male', '45000', '2017-02-08']
]

emp_schema = "employee_id string, department_id string, name string, age string, gender string, salary string, hire_date string"

In [0]:
emp = spark.createDataFrame(data=emp_data, schema=emp_schema)

In [0]:
emp.printSchema()

root
 |-- employee_id: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- hire_date: string (nullable = true)



In [0]:
# Get unique data from data frame
# select distinct emp.* from emp

emp_unique = emp.distinct().orderBy('employee_id')

In [0]:
emp_unique.show()

+-----------+-------------+---------------+---+------+------+----------+
|employee_id|department_id|           name|age|gender|salary| hire_date|
+-----------+-------------+---------------+---+------+------+----------+
|        001|          101|  Alice Johnson| 29|Female| 52000|2016-03-15|
|        002|          102|      Bob Smith| 34|  Male| 48000|2017-07-20|
|        003|          103|  Charlie Brown| 26|  Male| 47000|2018-02-01|
|        004|          104|   Diana Prince| 31|Female| 55000|2015-11-12|
|        005|          101|     Ethan Hunt| 28|  Male| 49500|2019-05-08|
|        006|          102|Fiona Gallagher| 27|Female| 51000|2016-09-10|
|        007|          103|George Costanza| 33|  Male| 46000|2015-01-01|
|        008|          108| Hannah Montana| 25|Female| 53000|2017-04-14|
|        009|          109|   Isaac Newton| 35|  Male| 60000|2018-10-25|
|        010|          110|  Jessica Jones| 32|Female| 58000|2016-06-30|
|        011|          111|   Kevin Malone| 36|  Ma

In [0]:
# unique of department_id
# select distinct department_id from emp

emp_dept_id = emp.select('department_id').distinct()

In [0]:
emp_dept_id.show()

+-------------+
|department_id|
+-------------+
|          101|
|          102|
|          104|
|          103|
|          108|
|          110|
|          109|
|          111|
|          115|
|          113|
|          107|
|          120|
+-------------+



In [0]:
# window functions
# select *, max(salary), over(partition by department_id order_by salay desc) as max_salary from emp_unique
from pyspark.sql.window import Window
from pyspark.sql.functions import max, col, desc, asc

window_spec = Window.partitionBy(col("department_id")).orderBy(col('salary').desc())
max_func = max(col('salary')).over(window_spec)

emp_1 = emp.withColumn('max_salary', max_func)

In [0]:
emp_1.show()

+-----------+-------------+---------------+---+------+------+----------+----------+
|employee_id|department_id|           name|age|gender|salary| hire_date|max_salary|
+-----------+-------------+---------------+---+------+------+----------+----------+
|        001|          101|  Alice Johnson| 29|Female| 52000|2016-03-15|     52000|
|        005|          101|     Ethan Hunt| 28|  Male| 49500|2019-05-08|     52000|
|        006|          102|Fiona Gallagher| 27|Female| 51000|2016-09-10|     51000|
|        002|          102|      Bob Smith| 34|  Male| 48000|2017-07-20|     51000|
|        003|          103|  Charlie Brown| 26|  Male| 47000|2018-02-01|     47000|
|        007|          103|George Costanza| 33|  Male| 46000|2015-01-01|     47000|
|        004|          104|   Diana Prince| 31|Female| 55000|2015-11-12|     55000|
|        017|          107|   Quinn Fabray| 26|Female| 51000|2015-11-05|     51000|
|        008|          108| Hannah Montana| 25|Female| 53000|2017-04-14|    

In [0]:
# window functions - 2nd highest salary of each department
# select *, row_number() over(partition by department_id order by salary desc) as rn from emp_unique where rn = 2
from pyspark.sql.window import Window
from pyspark.sql.functions import max, col, desc, asc, row_number

window_spec = Window.partitionBy(col("department_id")).orderBy(col('salary').desc())
# max_func = max(col('salary')).over(window_spec)
rn = row_number().over(window_spec)

# emp_1 = emp.withColumn('max_salary', max_func)
emp_2 = emp.withColumn('rn', rn).where('rn = 2')


In [0]:
emp_2.show()

+-----------+-------------+---------------+---+------+------+----------+---+
|employee_id|department_id|           name|age|gender|salary| hire_date| rn|
+-----------+-------------+---------------+---+------+------+----------+---+
|        005|          101|     Ethan Hunt| 28|  Male| 49500|2019-05-08|  2|
|        002|          102|      Bob Smith| 34|  Male| 48000|2017-07-20|  2|
|        007|          103|George Costanza| 33|  Male| 46000|2015-01-01|  2|
|        019|          109|   Sophia Loren| 30|Female| 56000|2018-09-17|  2|
|        010|          110|  Jessica Jones| 32|Female| 58000|2016-06-30|  2|
|        013|          111|      Mike Ross| 29|  Male| 51500|2019-01-05|  2|
|        015|          115| Oscar Martinez| 28|      | 45500|2018-03-19|  2|
|        020|          120|Toby Flenderson| 35|  Male| 45000|2017-02-08|  2|
+-----------+-------------+---------------+---+------+------+----------+---+



In [0]:
# window functions using expr
# select *, row_number() over(partition by department_id order by salary desc) as rn from emp_unique where rn = 2
from pyspark.sql.functions import expr

emp_3 = emp.withColumn('rn', expr("row_number() over(partition by department_id order by salary desc)")).where('rn = 2')


In [0]:
emp_3.show()

+-----------+-------------+---------------+---+------+------+----------+---+
|employee_id|department_id|           name|age|gender|salary| hire_date| rn|
+-----------+-------------+---------------+---+------+------+----------+---+
|        005|          101|     Ethan Hunt| 28|  Male| 49500|2019-05-08|  2|
|        002|          102|      Bob Smith| 34|  Male| 48000|2017-07-20|  2|
|        007|          103|George Costanza| 33|  Male| 46000|2015-01-01|  2|
|        019|          109|   Sophia Loren| 30|Female| 56000|2018-09-17|  2|
|        010|          110|  Jessica Jones| 32|Female| 58000|2016-06-30|  2|
|        013|          111|      Mike Ross| 29|  Male| 51500|2019-01-05|  2|
|        015|          115| Oscar Martinez| 28|      | 45500|2018-03-19|  2|
|        020|          120|Toby Flenderson| 35|  Male| 45000|2017-02-08|  2|
+-----------+-------------+---------------+---+------+------+----------+---+

